# Import libraries

In [ ]:
! pip install pandas
! pip install numpy
! pip install scikit-learn

In [16]:
import pandas as pd
import numpy as np
import sys
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
from sklearn.metrics import balanced_accuracy_score

# Set constants

In [17]:
REGULARIZATION = 0.03
PRESENCE_CUTOFF = 0.00001

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

# Create helper functions and classes

In [18]:
if not sys.warnoptions:
    warnings.simplefilter("ignore")

class GMWI(BaseEstimator):

    def __init__(self, use_shannon=False, theta_f=1, theta_d=0):
        self.use_shannon = use_shannon
        self.fitted = False
        self.thresh = 0.00001
        self.health_abundant = None
        self.health_scarce = None
        self.features = None
        self.theta_f = theta_f
        self.theta_d = theta_d

    def fit(self, X, y):
        """
        Identifies health_abundant and health_scarce
        columns/features
        """
        self.features = X.columns
        self.classes_ = np.unique(y)
        if(isinstance(X, pd.DataFrame)):
            X = X.values
        if(isinstance(y, pd.DataFrame)):
            y = y.values
        self.fitted = True
        difference, fold_change = self.get_proportion_comparisons(X, y)
        self.select_features(difference, fold_change)

    def get_proportion_comparisons(self, X, y):
        # get healthy and unhealthy samples
        healthies = X[y.flatten(), :]
        unhealthies = X[~y.flatten(), :]

        # get proportions for each species
        proportion_healthy = self.get_proportions(healthies)
        proportion_unhealthy = self.get_proportions(unhealthies)

        # get differences and fold change
        diff = proportion_healthy - proportion_unhealthy
        fold = proportion_healthy / proportion_unhealthy
        return diff, fold

    def get_proportions(self, samples_of_a_class):
        num_samples = samples_of_a_class.shape[0]
        p = np.sum(samples_of_a_class > self.thresh, axis=0) / num_samples
        return p

    def select_features(self, difference, fold_change):
        # based on proportion differences and fold change, select health abundant
        # and health scarce
        self.health_abundant = self.features[self.cutoff(difference, fold_change)]
        self.health_scarce = self.features[self.cutoff(-1 * difference, 1 / fold_change)]

    def cutoff(self, diff, fold):
        diff_cutoff = diff > self.theta_d
        fold_cutoff = fold > self.theta_f
        both_cutoff = np.bitwise_and(diff_cutoff, fold_cutoff)
        columns = np.where(both_cutoff)
        return columns[0]

    def decision_function(self, X):
        if not self.fitted:
            return None
        if list(X.columns) != list(self.features):
            raise Exception("Model was trained with (different) feature names than input")
        # if(isinstance(X, pd.DataFrame)):
        #    X = X.values
        X_healthy_features = X[self.health_abundant]
        X_unhealthy_features = X[self.health_scarce]
        psi_MH = self.get_psi(X_healthy_features.values) / (
            X_healthy_features.shape[1])
        psi_MN = self.get_psi(X_unhealthy_features.values) / (
            (X_unhealthy_features.shape[1]))
        num = psi_MH + self.thresh
        dem = psi_MN + self.thresh
        return np.log10(num / dem)

    def get_psi(self, X):
        psi = self.richness(X) * 1.0
        if self.use_shannon:
            shan = self.shannon(X)
            psi *= shan
        return psi

    def richness(self, X):
        """
        Returns the number of nonzero values for each sample (row) in X
        """
        rich = np.sum(X > self.thresh, axis=1)
        return rich

    def shannon(self, X):
        logged = np.log(X)
        logged[logged == -np.inf] = 0
        logged[logged == np.inf] = 0
        shan = logged * X * -1
        return np.sum(shan, axis=1)

    def predict(self, X):
        return self.decision_function(X) > 0

# Load up data

In [19]:
# download the zipped data if we're in colab
! $IN_COLAB && wget https://raw.githubusercontent.com/danielchang2002/GMWI2/main/manuscript/data.zip
    
# unzip the data
! unzip data.zip

unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


In [20]:
training_set = pd.read_csv("data/training_set.csv", index_col=[0, 1])

In [21]:
X = training_set.iloc[:, :-1]
X

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID            Sample Accession                                            
Obregon-Tito (2015) SAMN03283239        0.000000                     0.000000   
                    SAMN03283266        0.000000                     0.000000   
                    SAMN03283281        0.009764                     0.009764   
                    SAMN03283294        0.000000                     0.000000   
                    SAMN03283288        0.011865                     0.011865   
...                                          ...                          ...   
Yang (2020)         SRR6456373          0.000000                     0.000000   
                    SRR6456374          0.000000                     0.000000   
                    SRR6456375          0.000000                     0.000000   
                    SRR6456376          0.002415                     0.002415   
                    SRR6456377          0.000000                     0.000000   

                                      k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID            Sample Accession                                                   
Obregon-Tito (2015) SAMN03283239                                            0.000000   
                    SAMN03283266                                            0.000000   
                    SAMN03283281                                            0.009764   
                    SAMN03283294                                            0.000000   
                    SAMN03283288                                            0.011865   
...                                                                              ...   
Yang (2020)         SRR6456373                                              0.000000   
                    SRR6456374                                              0.000000   
                    SRR6456375                                              0.000000   
                    SRR6456376                                              0.002415   
                    SRR6456377                                              0.000000   

                                      k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID            Sample Accession                                                                         
Obregon-Tito (2015) SAMN03283239                                               0.000000                      
                    SAMN03283266                                               0.000000                      
                    SAMN03283281                                               0.009764                      
                    SAMN03283294                                               0.000000                      
                    SAMN03283288                                               0.011865                      
...                                                                                 ...                      
Yang (2020)         SRR6456373                                                 0.000000                      
                    SRR6456374                                                 0.000000                      
                    SRR6456375                                                 0.000000                      
                    SRR6456376                                                 0.002415                      
                    SRR6456377                                                 0.000000                      

                                      k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID            Sample Accession                                                                                                
Obregon-Tito (2015) SAMN03283239                                               0.000000                                             
         

In [22]:
y = training_set.iloc[:, [-1]]
y

is_healthy
Study_ID            Sample Accession            
Obregon-Tito (2015) SAMN03283239            True
                    SAMN03283266            True
                    SAMN03283281            True
                    SAMN03283294            True
                    SAMN03283288            True
...                                          ...
Yang (2020)         SRR6456373              True
                    SRR6456374              True
                    SRR6456375              True
                    SRR6456376              True
                    SRR6456377              True

[8069 rows x 1 columns]

# Train GMWI2 and GMWI and evaluate on training set

In [23]:
# use same parameters as original paper
gmwi = GMWI(use_shannon=True, theta_f=1.4, theta_d=0.1)

# use only non-viral species
X_GMWI = X.copy()
X_GMWI = X_GMWI[list(filter(lambda x : "s__" in x and "virus" not in x, X_GMWI.columns))]

gmwi.fit(X_GMWI, y)
GMWI_scores = pd.DataFrame(gmwi.decision_function(X_GMWI), index=y.index, columns=["GMWI"])
display(GMWI_scores)
print("GMWI balanced_accuracy on training set:", balanced_accuracy_score(y, GMWI_scores > 0))

GMWI
Study_ID            Sample Accession          
Obregon-Tito (2015) SAMN03283239      3.832094
                    SAMN03283266      1.758548
                    SAMN03283281      3.341858
                    SAMN03283294      1.429055
                    SAMN03283288      1.064592
...                                        ...
Yang (2020)         SRR6456373       -4.472053
                    SRR6456374       -2.158801
                    SRR6456375        0.280615
                    SRR6456376        1.132266
                    SRR6456377       -0.305132

[8069 rows x 1 columns]

GMWI balanced_accuracy on training set: 0.7172501242714733


In [24]:
gmwi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=REGULARIZATION, class_weight="balanced")
X_GMWI2 = X > PRESENCE_CUTOFF
gmwi2.fit(X_GMWI2, y.values.flatten())
GMWI2_scores = pd.DataFrame(gmwi2.decision_function(X_GMWI2), index=y.index, columns=["GMWI2"])
display(GMWI2_scores)
print("GMWI2 balanced_accuracy on training set:", balanced_accuracy_score(y, GMWI2_scores > 0))

GMWI2
Study_ID            Sample Accession          
Obregon-Tito (2015) SAMN03283239      1.835237
                    SAMN03283266      0.103722
                    SAMN03283281      1.116329
                    SAMN03283294      1.758818
                    SAMN03283288      1.675791
...                                        ...
Yang (2020)         SRR6456373       -1.029233
                    SRR6456374       -0.941967
                    SRR6456375       -0.030621
                    SRR6456376        0.973798
                    SRR6456377       -0.066339

[8069 rows x 1 columns]

GMWI2 balanced_accuracy on training set: 0.7988526637127441


# Apply GMWI2 to longitudinal case studies

### Tanes *et al.* 2021 (dietary fiber)

In [65]:
pd.read_csv("data/metadata_val.csv")

,Study_ID,Sample Accession,Subject_ID,health_status/host_diet,timepoint
0,P65,SAMEA104062441,ERAS10,Healthy,D0
1,P65,SAMEA104062442,ERAS10,Healthy,D180
2,P65,SAMEA104062443,ERAS10,Healthy,D42
3,P65,SAMEA104062444,ERAS10,Healthy,D8
4,P65,SAMEA104062445,ERAS11,Healthy,D0
...,...,...,...,...,...
1358,P90,SAMN16701308,9003,EEN,3
1359,P90,SAMN16701460,9017,Vegan,1
1360,P90,SAMN16701459,9016,EEN,15
1361,P90,SAMN16701458,9016,EEN,14
